In [ ]:
import os
import numpy as np
import TissueParameterList as TPL

from pathlib import Path
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"

fpath_input = root_path / "Input/"

fname_xml = fpath_input / "XCAT_TissueParameters_XML.xml"
tpl = TPL.TissueParameterList()
tpl.parse_xml(str(fname_xml))
label_name_dict = tpl.get_labels_and_names()


In [ ]:
from scipy.ndimage.morphology import binary_erosion
import matplotlib.pyplot as plt
def get_mask_for_labels(segmentation, labels):
    mask = np.zeros(segmentation.shape, dtype=bool)
    for lab in labels:
        mask += (np.round(segmentation) == lab)
    mask = binary_erosion(mask)
    return mask

def get_myocard_mask(segmentation):
    labels_myocard = [1,2,3,4]
    return get_mask_for_labels(segmentation, labels_myocard)


def load_data(fname):
    
    print("Loading {}".format(fname))
    data = np.load(fname)
    
    labels= data["labels"]
    gt_t1 = data["gt_t1"]
    gt_t2 = data["gt_t2"]
    
    match_t1 = data["match_t1"]
    match_t2 = data["match_t2"]
    
    mask = get_myocard_mask(labels)
    
    return gt_t1, gt_t2, match_t1, match_t2, mask, labels
    

def analyse_myocardium(t1map, t2map, mask):
    
    t1 = t1map[mask]
    t2 = t2map[mask]
    
    return t1, t2


In [ ]:

def analyse_dataset(fpath_data, i):
    
    gt_t1, gt_t2, match_t1, match_t2, mask, labels = load_data(fpath_data)

    f,ax = plt.subplots(2,2)
    ax[0,0].imshow(gt_t1,cmap='jet', vmin=0, vmax=2500)
    ax[0,0].axis("off")

    ax[1,0].imshow(gt_t2,cmap='magma', vmin=10, vmax=150)
    ax[1,0].axis("off")

    ax[0,1].imshow(match_t1,cmap='jet', vmin=0, vmax=2500)
    ax[0,1].axis("off")

    ax[1,1].imshow(match_t2,cmap='magma', vmin=10, vmax=150)
    ax[1,1].axis("off")
    
    ax[0,0].set_title("run num {}".format(i))
    

    t1_gt_val, t2_gt_val = analyse_myocardium(gt_t1, gt_t2, mask)
    match_t1_val, match_t2_val = analyse_myocardium(match_t1, match_t2, mask)
    
    return t1_gt_val, t2_gt_val, match_t1_val, match_t2_val


In [ ]:
import numpy as np

parameters = ["T1", "T2"]
fittype = ["GT", "Fit"]
motion = ["stat", "resp", "card", "cardioresp", "half resp", "card gated"]


mean_values = np.zeros(shape=(len(parameters),len(fittype),len(motion)), dtype=object)
std_values = np.zeros(shape=(len(parameters),len(fittype),len(motion)), dtype=object)


In [ ]:
def store_stat_to_values(fname_data, mv, sv, idx):
    
    res = analyse_dataset(fname_data, idx)

    mv[0,0,idx] = np.round(np.nanmean(res[0]))
    mv[1,0,idx] = np.round(np.nanmean(res[1]))
    mv[0,1,idx] = np.round(np.nanmean(res[2]))
    mv[1,1,idx] = np.round(np.nanmean(res[3]))

    sv[0,0,idx] = np.round(np.nanstd(res[0]))
    sv[1,0,idx] = np.round(np.nanstd(res[1]))
    sv[0,1,idx] = np.round(np.nanstd(res[2]))
    sv[1,1,idx] = np.round(np.nanstd(res[3]))


fpath_data = root_path / "Output/output_i_fitresults_stat.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 0)

fpath_data = root_path / "Output/output_i_fitresults_resp.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 1)

fpath_data = root_path / "Output/output_i_fitresults_card.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 2)

fpath_data = root_path / "Output/output_i_fitresults_cardioresp.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 3)

fpath_data = root_path / "Output/output_i_fitresults_half_resp.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 4)

fpath_data = root_path / "Output/output_i_fitresults_gated_card.npz"
store_stat_to_values(fpath_data, mean_values, std_values, 5)


In [ ]:
import xarray as xr

meandata = xr.DataArray(data=mean_values, coords=[parameters,fittype,motion], dims=["parameter", "fittype", "motion"])
stddata = xr.DataArray(data=std_values, coords=[parameters,fittype,motion], dims=["parameter", "fittype", "motion"])


In [ ]:
t1dat = meandata.sel(parameter='T1').values
t1err = stddata.sel(parameter='T1').values

x = np.array([0,1])
f = plt.figure()
for i in range(t1dat.shape[1]):
    ct1 = t1dat[:,i]
    et1 = t1err[:,i]
    plt.errorbar(x=x, y=ct1, yerr=et1, marker='o', label=motion[i])

plt.xlabel("Fit Type")
plt.ylabel("T1 (ms)")
plt.legend()

tick_locs = (0, 1)
tick_labels = ('GT', 'Fit')
plt.xticks(tick_locs, tick_labels)
plt.savefig("/media/sf_CCPPETMR/motion_analysis_T1.png", dpi=300)

In [ ]:
t1dat = meandata.sel(parameter='T2').values
t1err = stddata.sel(parameter='T2').values

x = np.array([0,1])
f = plt.figure()
for i in range(t1dat.shape[1]):
    ct1 = t1dat[:,i]
    et1 = t1err[:,i]
    plt.errorbar(x=x, y=ct1, yerr=et1, marker='o', label=motion[i])

plt.xlabel("Fit Type")
plt.ylabel("T2 (ms)")
plt.legend()

tick_locs = (0, 1)
tick_labels = ('GT', 'Fit')
plt.xticks(tick_locs, tick_labels)
plt.savefig("/media/sf_CCPPETMR/motion_analysis_T2.png", dpi=300)

In [ ]:

import pandas as pd
mean_frame = pd.DataFrame(meandata.to_series())
std_frame =  pd.DataFrame(stddata.to_series())




data = pd.concat([mean_frame, std_frame], axis=1)

def write_txt(fname, the_text):
    f = open(fname, 'w')
    f.write(the_text)
    f.close()

write_txt('/media/sf_CCPPETMR/parameter_evaluation.tex', data.to_latex())

# plt.legend()